In [1]:
import sqlitedatastore as datastore
import triematcher as matcher
from annoutil import find_xs_in_y


datastore.connect()
dic_positive, dic_negative = matcher.get_sentiment_dictionaries()
doc_id = 1
for sent in datastore.get_annotation(doc_id, 'sentence'):
    tokens = find_xs_in_y(
        datastore.get_annotation(doc_id, 'token'), sent)
    text = ''.join([token['lemma'] for token in tokens])
    print(text, '-->')
    print('\tpositive:', matcher.search_terms(text, dic_positive))
    print('\tnegative:', matcher.search_terms(text, dic_negative))
datastore.close()


フィジー -->
	positive: []
	negative: []
フィジー共和国（フィジーきょうわこく）、通称フィジーは、オセアニアの国家で、イギリス連邦加盟国だある -->
	positive: [('和', 5), ('こく', 16), ('通', 20)]
	negative: []
首都は*島のスバ -->
	positive: []
	negative: []
南太平洋のフィジー諸島と北に**程離れるた保護領の*島に位置する群島国家だある -->
	positive: [('太平', 1), ('保護', 21)]
	negative: []
*余の火山島と珊瑚礁からなる -->
	positive: []
	negative: []
西にバヌアツ、東にトンガ、北にツバルがある -->
	positive: []
	negative: []
目次 -->
	positive: []
	negative: []
国名*編集* -->
	positive: [('名', 1)]
	negative: []
正式名称は、***（*） -->
	positive: [('正', 0), ('名', 2)]
	negative: []
通称、* -->
	positive: [('通', 0)]
	negative: []
国名の由来には諸説ある、説として有力だのは前者だある -->
	positive: [('名', 1), ('有力', 16), ('力', 17)]
	negative: []
日本語の表記は、フィジー共和国 -->
	positive: [('和', 13)]
	negative: []
通称、フィジー -->
	positive: [('通', 0)]
	negative: []
他に、*、* -->
	positive: []
	negative: []
*年にフィジー共和国からフィジー諸島共和国へと改称するたが、*年*月に再度フィジー共和国に戻るた -->
	positive: [('和', 8), ('和', 19), ('和', 42)]
	negative: []
また、漢字による表記は「*」だある -->
	positive: []
	negative: []
歴史*編集* -->
	positive: []
	neg

In [2]:
import sqlitedatastore as datastore
import solrindexer as indexer


datastore.connect()
print('#label', 'doc_id', 'sentence_id', 'text')
results = indexer.search_annotation(
    fl_keyword_pairs=[
        ('sentence_txt_ja', [['教育', '治安', '経済']]),
        ('name_s',         [['sentence']]),
    ], rows=1000)
for r in results['response']['docs']:
    text = datastore.get(r['doc_id_i'], ['content'])['content']
    sent = datastore.get_annotation(r['doc_id_i'], 'sentence')[r['anno_id_i']]
    # ラベルファイルのデータ構造へ変換
    print(0, r['doc_id_i'], r['anno_id_i'], text[sent['begin']:sent['end']])
datastore.close()


#label doc_id sentence_id text
0 8 47 こうした経済混乱に、長期政権・一党支配に対する不満と相まって、治安の悪化も問題となっている
0 2 154 治安[編集]
0 3 719 治安維持
0 6 435 治安[編集]
0 2 155 ブラジルの治安は非常に悪い
0 3 721 国内の治安維持は、主に警察が担う
0 2 448 初等教育と中等教育（日本における高校以上の教育）、高等教育（日本における大学）からなり、初等教育と前期中等教育を併せた義務教育は8年間、後期中等教育は3年間となっている
0 8 42 ムガベ大統領は第二次コンゴ戦争への派兵に専念していったため、ジンバブエの経済や医療、教育などが悪化していった
0 2 153 「年金改革」と「汚職や犯罪との戦い」を掲げ、治安強化と軍政賛美の姿勢を見せている
0 3 190 1925年、男子普通選挙の成立と同時に制定された治安維持法は設立間もない日本共産党や社会主義勢力、後には自由主義なども広く弾圧した治安機関、特別高等警察の法的根拠となった
0 2 451 中等教育を終えると高等教育への道が開ける
0 8 164 6歳からの入学で初等教育7年、前期中等教育4年と後期中等教育2年、高等教育が3年程
0 11 737 インドの私立高校はすでに初等教育から英語で教育が行われている
0 12 650 コミュニケーションを重視した国語教育が、小学校での最重要の教育目標になってもいる
0 2 446 教育[編集]
0 3 876 教育段階
0 5 145 教育[編集]
0 6 433 教育[編集]
0 8 162 教育[編集]
0 10 361 教育[編集]
0 11 730 教育[編集]
0 12 646 教育[編集]
0 12 648 前期中等教育の後3年間のリセ（高等学校）による後期中等教育を経て高等教育への道が開ける
0 2 457 高等教育[編集]
0 3 873 教育・科学・技術
0 10 362 イギリスの学校教育は地域や公立・私立の別により異なるが、5歳より小学校教育が開始される
0 2 447 初等および中等教育[編集]
0 3 884 生涯学習・教育訓練
0 6 467 自治体の教育関係の予算は学区の税収と予算案に対する住民投票によって決定され、質の高い教育を提供できる

In [5]:
import time

from sklearn.externals import joblib

import mlclassifier
import sentimentclassifier
import sqlitedatastore as datastore
from annoutil import find_xs_in_y


datastore.connect()
# ラベル付きデータ読み込み
sentences = []
labels = []
with open('data/labels_sentiment.txt') as f:
    for line in f:
        if line.startswith('#'):
            continue
        d = line.rstrip().split()
        label, doc_id, sent_id = int(d[0]), d[1], int(d[2])
        sent = datastore.get_annotation(doc_id, 'sentence')[sent_id]
        tokens = find_xs_in_y(
            datastore.get_annotation(doc_id, 'token'), sent)
        sentences.append((doc_id, sent, tokens))
        labels.append(label)

# 学習データ特徴量生成
num_train = int(len(sentences) * 0.8)
sentences_train = sentences[:num_train]
labels_train = labels[:num_train]
features, vocab = sentimentclassifier.convert_into_features(sentences_train)

# 学習
time_s = time.time()
print(':::TRAIN START')
model = mlclassifier.train(labels_train, features)
print(':::TRAIN FINISHED', time.time() - time_s)

# 学習モデルをファイルに保存
joblib.dump(model, 'result/model_sentiment.pkl')
joblib.dump(vocab, 'result/vocab_sentiment.pkl')

# 分類の実行
features_test = sentimentclassifier.convert_into_features_using_vocab(
    sentences[num_train:], vocab)
predicteds = mlclassifier.classify(features_test, model)
for predicted, (doc_id, sent, tokens), label in zip(
        predicteds,
        sentences[num_train:],
        labels[num_train:]):
    # 結果の確認
    text = datastore.get(doc_id, ['content'])['content']
    if predicted == label:
        print('correct  ', '  ', label, predicted,
              text[sent['begin']:sent['end']])
    else:
        print('incorrect', '  ', label, predicted,
              text[sent['begin']:sent['end']])
datastore.close()


:::TRAIN START
:::TRAIN FINISHED 0.01635289192199707
incorrect    0 1 独立直後から旧宗主国だったポルトガルに代わって莫大なイギリスの投資を受け、「老いた母（ポルトガル）の代わりに金持ちの継母（イギリス）を得た」と表現されるほどの飛躍的な経済的発展を遂げた
incorrect    0 1 クビシェッキは積極的な外資導入などにより日本やアメリカなど諸外国での評価は高かったものの、ブラジリア建設と遷都などの強引な手法、そして経済的な混乱を招いたことに対してブラジル国内では現在でも賛否が大きく分かれている
incorrect    1 -1 安価な労働力と豊富な天然資源により、ブラジルは2004年度の国民総生産（GNP）で世界第9位に位置し、南半球および南アメリカの国家における最大の経済規模を有する
incorrect    1 -1 1979年以降の改革開放路線を皮切りに中国経済は急成長を続け、2010年に日本は中国に抜かれてGNPでソビエト連邦時代以来となる世界3位に後退した
incorrect    0 1 特に強調されるのは19世紀後半から20世紀初頭の日本の近代化に果たしたイギリスの役割であり、イギリスは経済・文化・学術・政治・軍事のあらゆる面において日本に最も強い影響力があった
incorrect    0 1 また、キューバなどの社会主義国とも経済・文化の両面で友好的な関係が築かれ、ペルー日本大使公邸占拠事件でも日本の要請を受けたキューバがゲリラの亡命受け入れを受諾するなど協力した
incorrect    -1 1 日本銀行は、2006年にゼロ金利を解除したが、未だ金利の水準が低く推移し、個人消費の伸びも見られないなど、経済回復が明確でなく、2007年現在、それ以上の金利引き上げに至っていない
incorrect    1 0 独立後四半世紀はカカオ、コーヒー、バナナなどの農産物、次いで1970年代後半採掘が始まった原油など第一次産品の輸出によって、アフリカ諸国の中でももっとも経済的に成功していた
correct      0 0 経済改革後、これまで寡占状態で家族経営が常態化し、政府との結びつきが続いていたインドの民間部門は外国との競争、とりわけ、中国製の安価な輸入品との競争に曝される